In [1]:
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import sklearn.datasets
import sklearn.linear_model
import matplotlib
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (10.0, 8.0)

print 'done'

//anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


done


In [31]:
import json

# 
def load_data():
    filename = "data_for_training.work"
    X_res = []
    y_res = []
    c1 = 0
    c0 = 0
    max_c = 10000
    for line in open(filename):
        row = json.loads(line.strip())
        label = row[-1]
        if label == 0:
            if c0 > max_c:
                continue
            else:
                c0 += 1
        elif label == 1:
            if c1 > max_c:
                continue
            else:
                c1 += 1
        X_res.append(row[:-1])
        y_res.append(label)
    return np.array(X_res, dtype='float64'), np.array(y_res, dtype='int')

X, y = load_data()
print 'raw data', len(y), np.sum(y)


from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print 'training & test dataset', X_train.shape, X_test.shape

print 'done'

raw data 14681 4680
training & test dataset (10276, 48) (4405, 48)
done


In [32]:
from sklearn import neural_network
import time, os, pickle

def get_clf(use_cache=True):
    filename = "nn.work.model"
    if use_cache and os.path.exists(filename):
        print 'cache hit'
        return pickle.loads(open(filename, 'r').read())
    clf = neural_network.MLPClassifier(hidden_layer_sizes=(20,), 
                                       activation='relu', 
                                       solver='adam', 
                                       alpha=0.03,  # 学习速率
                                       batch_size='auto', 
                                       learning_rate='constant', 
                                       learning_rate_init=0.001, 
                                       power_t=0.5, 
                                       max_iter=10000, 
                                       shuffle=True, 
                                       random_state=3,  # 初始状态的随机种子
                                       tol=0.0001, 
                                       verbose=False, 
                                       warm_start=False, 
                                       momentum=0.9,
                                       nesterovs_momentum=True, 
                                       early_stopping=False, 
                                       validation_fraction=0.1, 
                                       beta_1=0.9, 
                                       beta_2=0.999, 
                                       epsilon=1e-08)
    clf.fit(X_train, y_train)
    fout = open(filename, 'w')
    fout.write(pickle.dumps(clf))
    fout.close()
    return clf

clf = get_clf(False)

print 'iter:', clf.n_iter_, 'layers:', clf.n_layers_
t1 = time.time()
proba = clf.predict_proba(X_test)[:,1]  # 每个元素都是属于该类别的概率大小
print 'time:', time.time() - t1, 'size:', len(X_test)

print 'score of train:test', clf.score(X_train, y_train), clf.score(X_test, y_test)


iter: 156 layers: 3
time: 0.00309991836548 size: 4405
score of train:test 0.8369015181 0.826560726447


In [33]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=23, 
                             criterion='gini', 
                             max_depth=None, 
                             min_samples_split=2,
                             min_samples_leaf=1, 
                             min_weight_fraction_leaf=0.0, 
                             max_features='auto', 
                             max_leaf_nodes=None, 
                             min_impurity_split=1e-07, bootstrap=True, oob_score=False, 
                             n_jobs=1, random_state=None, verbose=0, warm_start=False, 
                             class_weight=None)

clf.fit(X_train, y_train)

t1 = time.time()
clf.predict(X_test)
print 'time:', time.time() - t1, 'size:', len(X_test)

print clf.score(X_train, y_train), clf.score(X_test, y_test)


time: 0.0320601463318 size: 4405
0.9980537174 0.845856980704


In [ ]:
from sklearn import linear_model
from sklearn import svm
from sklearn.svm import LinearSVC

lr = linear_model.LogisticRegressionCV()
svc = svm.SVC(kernel='rbf')

#X_train = [[1,0,0],[0,1,0],[1,1,0],[0,0,1],[0,1,1],[1,0,1]]
#y_train = [0,0,0,1,1,1]

def get_err_rate(X, y, clf):
    res = clf.predict(X)
    total_count = len(res)
    wrong_count = np.sum(np.abs(res - y))
    return float(wrong_count) / total_count

max_acc = 0
best_C = 0
for C in np.linspace(75, 85, 100):
    clf = svm.SVC(kernel='rbf', C=C)
    clf.fit(X, y)
    clf.fit(X_train, y_train)
    acc_rate = clf.score(X_train, y_train)
    if acc_rate > max_acc:
        max_acc = acc_rate
        best_C = C
        print best_C, max_acc

clf = svm.SVC(C=best_C)
clf.fit(X_train, y_train)
print X_train.shape, y_train.shape, X_test.shape, y_test.shape
print 'acc', clf.score(X_test, y_test)
#print get_err_rate(X_test, y_test, lsvc)

print 'done'